In [1]:
import h5py, numpy as np

In [2]:
with h5py.File('silicon_fullscan_sparse.h5','r') as hin:
    def v(n,x):
        if type(x) is h5py.Dataset:
            print(n,x.shape, x.dtype)
        else:
            print(n, type(x))
    hin.visititems(v)
    s = hin['1.1']
    col = s['col'][:]
    row = s['row'][:]
    frame = s['frame'][:]
    nnz=s['nnz'][:]
    intensity=s['intensity'][:]

1.1 <class 'h5py._hl.group.Group'>
1.1/col (3626536,) uint16
1.1/frame (3626536,) uint32
1.1/instrument <class 'h5py._hl.group.Group'>
1.1/instrument/positioners <class 'h5py._hl.group.Group'>
1.1/instrument/positioners/dty () float64
1.1/instrument/positioners/px () float64
1.1/instrument/positioners/py () float64
1.1/instrument/positioners/pz () float64
1.1/instrument/positioners/rot (180500,) float64
1.1/instrument/positioners/shtx () float64
1.1/instrument/positioners/shty () float64
1.1/instrument/positioners/shtz () float64
1.1/intensity (3626536,) uint16
1.1/measurement <class 'h5py._hl.group.Group'>
1.1/measurement/rot (180500,) float64
1.1/measurement/rot_center (180500,) float64
1.1/nnz (180500,) uint32
1.1/row (3626536,) uint16


In [3]:
#import pylab as pl, numpy as np
#pl.hist2d( col, np.log(intensity), bins=(1000,32), norm='log');

In [4]:
mask = intensity > 9
snew = { '1.1/row' :   row[mask], 
        '1.1/intensity' : intensity[mask], 
        '1.1/col' : col[mask], 
        '1.1/frame' : frame[mask] }    
snew['1.1/nnz'] = np.bincount( snew['1.1/frame'], minlength = nnz.shape[0] )
print( nnz.shape, snew['1.1/nnz'].shape )

(180500,) (180500,)


In [5]:
frame.min(), frame.max(), nnz.shape

(4, 180465, (180500,))

In [6]:
skips = "1.1/instrument/positioners/rot", "1.1/measurement/rot",

with h5py.File('silicon_fullscan_sparse.h5','r') as hin:
    with h5py.File('silicon_fullscan_strong_sparse.h5','w') as hout:
        def v(n,x):
            if n in skips:
                return
            if type(x) is h5py.Group:
                hout.require_group( n )
                hout[n].attrs.update( hin[n].attrs )
            if type(x) is h5py.Dataset:
                if n in snew:
                    a = snew[n][()]
                else:
                    a = hin[n][()]
                if n.find('rot_center')>0:
                    a = np.round(a*1e5)/1e5
                if len(a.shape):
                    cmp = { 'compression' : 'gzip',  'compression_opts' : 9 }
                else:
                    cmp = {}
                hout.create_dataset( name = n,
                                     shape = a.shape,
                                     chunks= a.shape,
                                     dtype = a.dtype,
                                     data = a,
                                     ** cmp
                                      )
                hout[n].attrs.update( hin[n].attrs )
        hin.visititems( v )